In [3]:
import spikeinterface.full as si
import matplotlib.pyplot as plt
import numpy as np
import probeinterface as pi
from pathlib import Path
import os 
import pandas as pd 

global_job_kwargs = dict(n_jobs=4, chunk_duration="1s",progress_bar=True)
si.set_global_job_kwargs(**global_job_kwargs)


basefolder="D:/M8_SNA-135381_17092024_1_g0_imec1"

metapath = basefolder + str('/Meta')
if not os.path.isdir(metapath):
   os.makedirs(metapath)


recording =  si.read_spikeglx(basefolder, stream_id='imec1.ap', load_sync_channel=False)
lfp = si.read_spikeglx(basefolder, stream_id='imec1.lf', load_sync_channel=False)
event =  si.read_spikeglx(basefolder, stream_id='nidq', load_sync_channel=False)
print(recording)




SpikeGLXRecordingExtractor: 384 channels - 30.0kHz - 1 segments - 118,544,273 samples 
                            3,951.48s (1.10 hours) - int16 dtype - 84.79 GiB


In [37]:


bad_channel_ids, channel_labels = si.detect_bad_channels(lfp,method = 'coherence+psd',outside_channels_location = 'both')
names = lfp.channel_ids
depth = lfp.get_channel_locations()[:,1]


ar = pd.DataFrame({'name':names, 'depth':depth, 'labels':channel_labels})
ar.to_csv(metapath + str('/lfp_labels.csv'))
filtered_ar = ar[ar['labels'] == 'out']

print(filtered_ar)

C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\spikeinterface\preprocessing\detect_bad_channels.py:333: RuntimeWarning: invalid value encountered in divide
  xcorr = np.sum(raw * ref[:, np.newaxis], axis=0) / np.sum(ref**2)


               name   depth labels
293  imec0.lf#LF293  2920.0    out
294  imec0.lf#LF294  2940.0    out
295  imec0.lf#LF295  2940.0    out
296  imec0.lf#LF296  2960.0    out
297  imec0.lf#LF297  2960.0    out
..              ...     ...    ...
379  imec0.lf#LF379  3780.0    out
380  imec0.lf#LF380  3800.0    out
381  imec0.lf#LF381  3800.0    out
382  imec0.lf#LF382  3820.0    out
383  imec0.lf#LF383  3820.0    out

[91 rows x 3 columns]


C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\spikeinterface\preprocessing\detect_bad_channels.py:216: UserWarning: Over 1/3 of channels are detected as bad. In the presence of a highnumber of dead / noisy channels, bad channel detection may fail (good channels may be erroneously labeled as dead).
  warnings.warn(


In [2]:
fig, ax = plt.subplots(figsize=(15, 15))
si.plot_probe_map(recording, ax=ax, with_channel_ids=True)
ax.set_ylim(-200,200)

NameError: name 'plt' is not defined

In [5]:
rec1 = si.highpass_filter(recording, freq_min=400.)
rec1 = si.phase_shift(rec1)
bad_channel_ids, channel_labels = si.detect_bad_channels(rec1,method = 'coherence+psd')
print(bad_channel_ids)
rec1 = si.interpolate_bad_channels(recording=rec1, bad_channel_ids=bad_channel_ids)

rec1 = si.common_reference(rec1, operator="median", reference="global")
print(rec1)


%matplotlib widget
si.plot_traces({'raw':recording,'filtered':rec1}, backend='ipywidgets')

['imec0.ap#AP191']
CommonReferenceRecording: 384 channels - 30.0kHz - 1 segments - 66,465,367 samples 
                          2,215.52s (36.93 minutes) - int16 dtype - 47.54 GiB


AppLayout(children=(TimeSlider(children=(Dropdown(description='segment', options=(0,), value=0), Button(icon='…

In [4]:


def extract_and_save_ttl_events(data, bits, save_path):
    digital_signals = data.get_traces()
    digital_word = digital_signals[:, 0]
    sampling_rate = data.get_sampling_frequency()
    for bit in bits:
        # Extract TTL pulses for the current bit
        ttl_timestamps = extract_ttl_from_bit(digital_word, bit, sampling_rate)
        
        ttl_df = pd.DataFrame(ttl_timestamps, columns=['timestamps'])
        
        filename = f'ttl_{bit}.csv'
        
        ttl_df.to_csv(f"{save_path}/{filename}", index=False)
        print(f"Extracted TTL event timestamps for bit {bit} saved to {filename}")


def extract_ttl_from_bit(digital_word, bit, sampling_rate):
    # Extract the specific bit from the word (bit-shifting and masking)
    ttl_signal = (digital_word >> bit) & 1  # Right shift and mask to isolate the specific bit
    
    # Detect rising edges (0 -> 1 transitions)
    ttl_rising_edges = np.where(np.diff(ttl_signal) > 0)[0]
    
    # Convert sample indices to timestamps (in seconds)
    ttl_timestamps = ttl_rising_edges / sampling_rate
    
    return ttl_timestamps


bits_to_extract = [0, 1, 2]  
extract_and_save_ttl_events(event , bits_to_extract, metapath)

Extracted TTL event timestamps for bit 0 saved to ttl_0.csv
Extracted TTL event timestamps for bit 1 saved to ttl_1.csv
Extracted TTL event timestamps for bit 2 saved to ttl_2.csv


In [ ]:
from spikeinterface.sorters import installed_sorters
installed_sorters()
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
torch.cuda.get_device_name(0)

Sorting_KS4 = si.run_sorter(sorter_name="kilosort4", recording=rec1, folder=basefolder + str('/sorted'),remove_existing_folder=True)

True
0
Loading recording with SpikeInterface...
number of samples: 66465367
number of channels: 384
numbef of segments: 1
sampling rate: 29999.928571428572
dtype: int16


In [21]:
#Sorting_KS4 = si.read_kilosort(folder_path=basefolder + str('/sorted/sorter_output'))
analyzer = si.create_sorting_analyzer(Sorting_KS4, rec1, sparse=True, format="memory")



estimate_sparsity:   0%|          | 0/901 [00:00<?, ?it/s]

In [22]:
analyzer.compute(['random_spikes', 'waveforms', 'templates', 'noise_levels','unit_locations','correlograms'],**global_job_kwargs)
analyzer.compute('spike_amplitudes')
analyzer.compute('principal_components', n_components = 5, mode="by_channel_local",**global_job_kwargs)

compute_waveforms:   0%|          | 0/901 [00:00<?, ?it/s]

spike_amplitudes:   0%|          | 0/901 [00:00<?, ?it/s]

Fitting PCA:   0%|          | 0/551 [00:00<?, ?it/s]

Projecting waveforms:   0%|          | 0/551 [00:00<?, ?it/s]

In [27]:
metric_names=['firing_rate', 'presence_ratio', 'snr','isi_violation', 'amplitude_cutoff']
metrics = si.compute_quality_metrics(analyzer, metric_names=metric_names)


amplitude_cutoff_thresh = 0.1
isi_violations_ratio_thresh = 0.5
presence_ratio_thresh = 0.9


our_query = f"(amplitude_cutoff < {amplitude_cutoff_thresh}) & (isi_violations_ratio < {isi_violations_ratio_thresh}) & (presence_ratio > {presence_ratio_thresh})"

keep_units = metrics.query(our_query)
keep_unit_ids = keep_units.index.values
analyzer_clean = analyzer.select_units(keep_unit_ids, folder=basefolder +str('/analyzer_clean'), format='binary_folder')
print(analyzer)
print(analyzer_clean)


C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\spikeinterface\qualitymetrics\misc_metrics.py:1273: UserWarning: Amplitude PDF does not have a unique minimum! More spikes might be required for a correct amplitude_cutoff computation!
  warnings.warn(
C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\spikeinterface\qualitymetrics\misc_metrics.py:902: UserWarning: Some units have too few spikes : amplitude_cutoff is set to NaN
  warnings.warn(f"Some units have too few spikes : amplitude_cutoff is set to NaN")


SortingAnalyzer: 384 channels - 551 units - 1 segments - memory - sparse - has recording
Loaded 9 extensions: random_spikes, waveforms, templates, noise_levels, unit_locations, correlograms, spike_amplitudes, principal_components, quality_metrics
SortingAnalyzer: 384 channels - 290 units - 1 segments - binary_folder - sparse - has recording
Loaded 9 extensions: random_spikes, waveforms, templates, noise_levels, unit_locations, correlograms, spike_amplitudes, principal_components, quality_metrics


In [1]:
si.plot_sorting_summary(sorting_analyzer=sorting_analyzer, curation=True, backend='spikeinterface_gui')

NameError: name 'si' is not defined

In [30]:
si.export_to_phy(analyzer_clean, output_folder=basefolder + str('/sorted/phy'),**global_job_kwargs)

write_binary_recording:   0%|          | 0/901 [00:00<?, ?it/s]

extract PCs:   0%|          | 0/901 [00:00<?, ?it/s]

Run:
phy template-gui  C:\Users\deepl\Desktop\Koen_Junli_optoelectrode_MLA-024623_03072024_g0_imec0\sorted\phy\params.py


In [35]:
!phy template-gui  C:\Users\deepl\Desktop\Koen_Junli_optoelectrode_MLA-024623_03072024_g0_imec0\sorted\phy\params.py

18:43:00.048 [W] model:667            Skipping spike waveforms that do not exist, they will be extracted on the fly from the raw data as needed.
